In [6]:
import os

In [7]:
Data_Dir = os.listdir("Images")

In [8]:
Data_Dir

['without_mask', 'with_mask']

In [9]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [10]:
img_rows, img_cols =112,112

In [11]:
images = []
labels = []

In [12]:
for category in Data_Dir:
    print(category)
    folder_path = os.path.join("Images",category)
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path,img)
        img = cv2.imread(img_path)
        
        try:
            grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            resized_img = cv2.resize(grayscale_img,(img_rows,img_cols))
            images.append(resized_img)
            labels.append(category)
        except Exception as e:
            print('Exception:',e)
            
images = np.array(images)/255.0
images = np.reshape(images,(images.shape[0],img_rows,img_cols,1))

without_mask
with_mask


In [13]:
labels[-1]

'with_mask'

In [14]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels = np.array(labels)

In [15]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [16]:
(train_x,test_x,train_y,test_y) = train_test_split(images,labels,test_size=0.25,
                                                  random_state=0)

# MODEL CREATION

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D

num_classes = 2
batch_size = 32

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 110, 110, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 53, 53, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 86528)             0

In [78]:
from tensorflow.keras.utils.vis_utils import plot_model
plot_model(model, to_file='face_mask_detection_architecture.png')

ModuleNotFoundError: No module named 'tensorflow.keras.utils.vis_utils'

In [18]:
from tensorflow.keras.optimizers import Adam
epochs = 60
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(lr=0.0001),
             metrics = ['accuracy'])
fitted_model = model.fit(
    train_x,
    train_y,
    epochs=epochs,
    validation_split=0.25)

Train on 1610 samples, validate on 537 samples
Epoch 1/60
1610/1610 [==============================] - 44s 28ms/sample - loss: 0.6168 - acc: 0.6354 - val_loss: 0.5260 - val_acc: 0.7058
Epoch 2/60
1610/1610 [==============================] - 44s 28ms/sample - loss: 0.4889 - acc: 0.7435 - val_loss: 0.4845 - val_acc: 0.7356
Epoch 3/60
1610/1610 [==============================] - 45s 28ms/sample - loss: 0.3970 - acc: 0.8273 - val_loss: 0.3957 - val_acc: 0.8082
Epoch 4/60
1610/1610 [==============================] - 45s 28ms/sample - loss: 0.3254 - acc: 0.8677 - val_loss: 0.3329 - val_acc: 0.8547
Epoch 5/60
1610/1610 [==============================] - 45s 28ms/sample - loss: 0.2779 - acc: 0.8882 - val_loss: 0.2946 - val_acc: 0.8883
Epoch 6/60
1610/1610 [==============================] - 45s 28ms/sample - loss: 0.2379 - acc: 0.9143 - val_loss: 0.2694 - val_acc: 0.9032
Epoch 7/60
1610/1610 [==============================] - 45s 28ms/sample - loss: 0.2057 - acc: 0.9267 - val_loss: 0.2593 - val

In [19]:
model.save('face_mask_detection6.h5')

NameError: name 'history' is not defined

# Real time Detection

In [24]:


from tensorflow.keras.models import load_model
import cv2
import numpy as np
import smtplib

In [25]:
model = load_model('face_mask_detection6.h5')
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
vid_source = cv2.VideoCapture(0)
text_dict = {0:'Mask On',1:'No Mask'}
rect_color_dict = {0:(0,255,0),1:(0,0,255)}

while True:
    ret,img = vid_source.read()
    grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)
    
    for(x,y,w,h) in faces:
        face_img = grayscale_img[y:y+w,x:x+w]
        resized_img = cv2.resize(face_img,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result = model.predict(reshaped_img)
        label = np.argmax(result,axis=1)[0]
        print(result)
        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
        
#         if label==0:
#             print("mask")
#         else:
#             print("noMask")
    cv2.imshow("Mask Detector",img)
    key = cv2.waitKey(1)
    
    if key==27:
        break
vid_source.release()
cv2.destroyAllWindows()
source.release()

[[0.3940329 0.6059671]]
[[0.23940583 0.7605941 ]]
[[0.5598082  0.44019178]]
[[0.13186601 0.868134  ]]
[[0.13186601 0.868134  ]]
[[0.02440166 0.97559834]]
[[0.01133609 0.988664  ]]
[[0.01133609 0.988664  ]]
[[0.02967294 0.970327  ]]
[[0.11331447 0.8866855 ]]
[[0.11331447 0.8866855 ]]
[[0.09980743 0.90019256]]
[[0.01251001 0.98749   ]]
[[0.01251001 0.98749   ]]
[[0.24592237 0.7540777 ]]
[[0.04284982 0.9571502 ]]
[[0.04284982 0.9571502 ]]
[[0.01033092 0.989669  ]]
[[0.02869549 0.9713045 ]]
[[0.02869549 0.9713045 ]]
[[0.09743103 0.902569  ]]
[[0.5294232  0.47057676]]
[[0.41773888 0.58226115]]
[[0.01875988 0.9812401 ]]
[[0.01875988 0.9812401 ]]
[[0.05643996 0.94356   ]]
[[0.03759804 0.9624019 ]]
[[0.03759804 0.9624019 ]]
[[0.03303567 0.9669643 ]]
[[0.01867275 0.98132724]]
[[0.01867275 0.98132724]]
[[0.01307227 0.9869277 ]]
[[0.05446512 0.9455349 ]]
[[0.05253627 0.94746375]]
[[0.15107247 0.84892756]]
[[0.15107247 0.84892756]]
[[0.06890602 0.93109393]]
[[0.07890093 0.9210991 ]]
[[0.02315561 0

NameError: name 'source' is not defined

In [15]:
cv2.destroyAllWindows()


In [16]:
vid_source.release()


# DETECTION ON STILL IMAGES

In [2]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import smtplib

model = load_model('face_mask_detection6.h5')
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
text_dict = {0:'Mask On',1:'No Mask'}
rect_color_dict = {0:(0,255,0),1:(0,0,255)}

img = cv2.imread('yes.jpg')
grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)
print(faces)
for(x,y,w,h) in faces:
    face_img = grayscale_img[y:y+w,x:x+w]
    resized_img = cv2.resize(face_img,(112,112))
    normalized_img = resized_img/255.0
    reshaped_img = np.reshape(normalized_img,(1,112,112,1))
    result = model.predict(reshaped_img)
    print(result)
    label = np.argmax(result,axis=1)[0]
    cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
    cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
    cv2.putText(img,text_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
        
#         if label==0:
#             print("mask")
#         else:
#             print("noMask")
    cv2.imshow("Mask Detector",img)
key = cv2.waitKey(5000)
cv2.destroyAllWindows()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[[ 57  76 227 227]]
[[0.5043889 0.4956111]]
